In [1]:
import os
import json
from pathlib import Path

import pandas as pd

from neuralhydrology.nh_run import eval_run

In [2]:
# load allowed GPU ids
f = open("gpu.json")
gpus = json.load(f)
f.close()

In [3]:
epoch = 20
epoch_str = str(epoch) if epoch / 100 > 1 else f"0{epoch}"

In [4]:
run_dir = Path(f"./runs/{sorted(os.listdir('./runs'))[-1]}")

In [5]:
def eval_results(period: str, epoch: str = "010") -> pd.DataFrame:
    df = pd.read_csv(run_dir / f"{period}" / f"model_epoch{epoch}" / f"{period}_metrics.csv", dtype={'basin': str})
    df = df.set_index('basin')

    # Compute the median NSE from all basins, where discharge observations are available for that period
    print(f"Median NSE of the {period} period {df['NSE'].median():.3f}")
    print(f"Mean NSE of the {period} period {df['NSE'].mean():.3f}")
    
    return df

In [6]:
eval_run(run_dir, period="train", epoch=epoch, gpu=gpus[0])
df_train = eval_results("train", epoch=epoch_str)

# Evaluation:  19%|█▉        | 8/42 [00:06<00:28,  1.21it/s]

Basin 11397 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation:  40%|████      | 17/42 [00:12<00:16,  1.54it/s]

Basin 13002 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation:  43%|████▎     | 18/42 [00:12<00:16,  1.42it/s]

...

# Evaluation:  67%|██████▋   | 28/42 [00:23<00:16,  1.15s/it]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Evaluation:  83%|████████▎ | 35/42 [00:27<00:03,  2.12it/s]

Basin 19243 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation: 100%|██████████| 42/42 [00:31<00:00,  1.32it/s]
Median NSE of the train period 0.880
Mean NSE of the train period 0.814


In [7]:
eval_run(run_dir, period="validation", epoch=epoch, gpu=gpus[0])
df_val = eval_results("validation", epoch=epoch_str)

# Validation:  10%|▉         | 4/42 [00:00<00:04,  8.41it/s]

The following basins had not enough valid target values to calculate a standard deviation: 11163. NSE loss values for this basin will be NaN.


# Validation:  19%|█▉        | 8/42 [00:00<00:03,  8.75it/s]

Basin 11397 All simulated values are NaN, thus metrics will be NaN, too.


# Validation:  45%|████▌     | 19/42 [00:02<00:02, 10.22it/s]

...

# Validation:  60%|█████▉    | 25/42 [00:02<00:01,  9.76it/s]

Basin 13115 All simulated values are NaN, thus metrics will be NaN, too.


# Validation:  64%|██████▍   | 27/42 [00:03<00:01,  8.19it/s]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Validation: 100%|██████████| 42/42 [00:04<00:00,  8.77it/s]
Median NSE of the validation period 0.884
Mean NSE of the validation period 0.806


In [8]:
eval_run(run_dir, period="test", epoch=epoch, gpu=gpus[0])
df_test = eval_results("test", epoch=epoch_str)

# Evaluation:  10%|▉         | 4/42 [00:00<00:03,  9.66it/s]

The following basins had not enough valid target values to calculate a standard deviation: 11163. NSE loss values for this basin will be NaN.


# Evaluation:  17%|█▋        | 7/42 [00:00<00:04,  8.72it/s]

Basin 11397 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation:  45%|████▌     | 19/42 [00:02<00:02,  9.37it/s]

...

# Evaluation:  62%|██████▏   | 26/42 [00:02<00:01,  8.75it/s]

Basin 13128 All simulated values are NaN, thus metrics will be NaN, too.


# Evaluation:  67%|██████▋   | 28/42 [00:03<00:01,  8.18it/s]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Evaluation: 100%|██████████| 42/42 [00:04<00:00,  9.25it/s]
Median NSE of the test period 0.573
Mean NSE of the test period -3.819


In [9]:
df_train = df_train.rename(columns={"NSE": "NSE_train", "KGE": "KGE_train"})
df_val = df_val.rename(columns={"NSE": "NSE_val", "KGE": "KGE_val"})
df_test = df_test.rename(columns={"NSE": "NSE_test", "KGE": "KGE_test"})

In [10]:
df_tmp = pd.merge(df_train, df_val, left_index=True, right_index=True)
df = pd.merge(df_tmp, df_test, left_index=True, right_index=True)
df.head()

,NSE_train,KGE_train,NSE_val,KGE_val,NSE_test,KGE_test
basin,,,,,,
11001,0.869921,0.748137,0.798338,0.653017,0.905944,0.911571
11068,0.940313,0.939771,0.581811,0.644063,-0.250459,0.530885
11126,0.959912,0.966558,0.974560,0.984705,0.949990,0.966002
11129,0.930211,0.851709,0.949119,0.892957,0.923943,0.902098
11163,0.946528,0.959144,NaN,NaN,NaN,NaN


File will be saved in the current runs directory

In [11]:
df.to_csv(f"{str(run_dir)}/eval.csv")